In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt  
import torch
import torch.nn as nn
import torch.nn.functional as F



In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
print test_sentence

['When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', 'thy', "beauty's", 'field,', 'Thy', "youth's", 'proud', 'livery', 'so', 'gazed', 'on', 'now,', 'Will', 'be', 'a', "totter'd", 'weed', 'of', 'small', 'worth', 'held:', 'Then', 'being', 'asked,', 'where', 'all', 'thy', 'beauty', 'lies,', 'Where', 'all', 'the', 'treasure', 'of', 'thy', 'lusty', 'days;', 'To', 'say,', 'within', 'thine', 'own', 'deep', 'sunken', 'eyes,', 'Were', 'an', 'all-eating', 'shame,', 'and', 'thriftless', 'praise.', 'How', 'much', 'more', 'praise', "deserv'd", 'thy', "beauty's", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'of', 'mine', 'Shall', 'sum', 'my', 'count,', 'and', 'make', 'my', 'old', "excuse,'", 'Proving', 'his', 'beauty', 'by', 'succession', 'thine!', 'This', 'were', 'to', 'be', 'new', 'made', 'when', 'thou', 'art', 'old,', 'And', 'see', 'thy', 'blood', 'warm', 'when', 'thou', "feel'st", 'it', 'cold.']


In [5]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2])
           for i in range(len(test_sentence)-2)]
print trigram

[(('When', 'forty'), 'winters'), (('forty', 'winters'), 'shall'), (('winters', 'shall'), 'besiege'), (('shall', 'besiege'), 'thy'), (('besiege', 'thy'), 'brow,'), (('thy', 'brow,'), 'And'), (('brow,', 'And'), 'dig'), (('And', 'dig'), 'deep'), (('dig', 'deep'), 'trenches'), (('deep', 'trenches'), 'in'), (('trenches', 'in'), 'thy'), (('in', 'thy'), "beauty's"), (('thy', "beauty's"), 'field,'), (("beauty's", 'field,'), 'Thy'), (('field,', 'Thy'), "youth's"), (('Thy', "youth's"), 'proud'), (("youth's", 'proud'), 'livery'), (('proud', 'livery'), 'so'), (('livery', 'so'), 'gazed'), (('so', 'gazed'), 'on'), (('gazed', 'on'), 'now,'), (('on', 'now,'), 'Will'), (('now,', 'Will'), 'be'), (('Will', 'be'), 'a'), (('be', 'a'), "totter'd"), (('a', "totter'd"), 'weed'), (("totter'd", 'weed'), 'of'), (('weed', 'of'), 'small'), (('of', 'small'), 'worth'), (('small', 'worth'), 'held:'), (('worth', 'held:'), 'Then'), (('held:', 'Then'), 'being'), (('Then', 'being'), 'asked,'), (('being', 'asked,'), 'wher

In [6]:
vocb = set(test_sentence) # 通过set将重复的单词去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [16]:
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size*n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        #print emb,1
        emb = emb.view(1, -1)
        #print emb,2
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out)
        #print log_prob.size()
        return log_prob

ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

In [18]:
for epoch in range(100):
    print('epoch: {}'.format(epoch+1))
    print('*'*10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        #print word,'word'
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        print label,'label'
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss.data[0]
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))

epoch: 1
**********
Variable containing:
 88
[torch.LongTensor of size 1]
 label
Variable containing:
 24
[torch.LongTensor of size 1]
 label
Variable containing:
 83
[torch.LongTensor of size 1]
 label
Variable containing:
 90
[torch.LongTensor of size 1]
 label
Variable containing:
 1
[torch.LongTensor of size 1]
 label
Variable containing:
 56
[torch.LongTensor of size 1]
 label
Variable containing:
 13
[torch.LongTensor of size 1]
 label
Variable containing:
 68
[torch.LongTensor of size 1]
 label
Variable containing:
 44
[torch.LongTensor of size 1]
 label
Variable containing:
 72
[torch.LongTensor of size 1]
 label
Variable containing:
 90
[torch.LongTensor of size 1]
 label
Variable containing:
 52
[torch.LongTensor of size 1]
 label
Variable containing:
 7
[torch.LongTensor of size 1]
 label
Variable containing:
 21
[torch.LongTensor of size 1]
 label
Variable containing:
 55
[torch.LongTensor of size 1]
 label
Variable containing:
 91
[torch.LongTensor of size 1]
 label
Variab

In [28]:
word, label = trigram[3]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)
print predict_label.data[0]
predict_word = idx_to_word[int(predict_label.data[0])]
print('real word is {}, predict word is {}'.format(label, predict_word))


 90
[torch.LongTensor of size 1]



TypeError: int() argument must be a string or a number, not 'torch.LongTensor'